In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.externals import joblib  # Use 'import joblib' if you have a newer version

# Function to check data files and update or create them
def manage_data_files(file1_path, file2_path, entry):
    # Check for the existence of the data files
    try:
        data1 = pd.read_csv(file1_path)
    except FileNotFoundError:
        data1 = pd.DataFrame(columns=list(entry.keys())[:3] + ["Timestamp"])  # Q1, Q2, Q3

    try:
        data2 = pd.read_csv(file2_path)
    except FileNotFoundError:
        data2 = pd.DataFrame(columns=list(entry.keys())[3:] + ["Timestamp"])  # Q4 to Q10

    # Update the data files
    entry["Timestamp"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    new_row1 = entry[:3].copy()
    new_row1['Timestamp'] = entry['Timestamp']
    new_row2 = entry[3:].copy()
    new_row2['Timestamp'] = entry['Timestamp']
    
    data1 = data1.append(new_row1, ignore_index=True)
    data2 = data2.append(new_row2, ignore_index=True)

    data1.to_csv(file1_path, index=False)
    data2.to_csv(file2_path, index=False)

    return data1, data2

In [ ]:
# Function to load and apply the machine learning model
def apply_model(data, model_path):
    model = joblib.load(model_path)
    prediction = model.predict(data.drop(columns=['Timestamp']))
    if prediction == 1:
        return "Anomaly Detected, recommend to seek a doctor's advice if this hasn't been done prior."
    else:
        return "Nothing to worry about, no anomaly detected."

# Example entry for a new row (this should be received from the app interface)
entry_data = {
    "Q1": 30, "Q2": 160, "Q3": 55, "Q4": "Moderate", "Q5": "Regularly",
    "Q6": "Yes, mild cramps", "Q7": "No", "Q8": "Yes, mild symptoms",
    "Q9": "3-5 days", "Q10": "No"
}
entry = pd.Series(entry_data)

# File paths (should point to a location accessible on the user's phone)
file1_path = '/path/to/datafile1.csv'
file2_path = '/path/to/datafile2.csv'

# Manage data files
data1, data2 = manage_data_files(file1_path, file2_path, entry)

# Decide which model to use based on entries in data2
model_path = 'XG_model.pkl' if len(data2) >= 6 else 'svm_model.joblib'

# Apply the model
result_message = apply_model(data2, model_path)
print(result_message)
